In [1]:
import torch 
import torch.nn as nn
import torch.functional as F
import torchvision
import matplotlib.pyplot as plt 

In [ ]:
H, W, C = 224, 224, 3 # height x width x channels (3 - red, green, blue)
batch_size = 4096
P = 16 # resolution of a patch
N = H*W/P**2 # Number of patches 
D = 768 # constant latent layer
lr = 2e-4
max_iters = 1000000
n_layers = 12
num_heads = 12
head_size = D/num_heads

x = torch.randn((H, W, C)) # image representation 

In [ ]:
class ViT(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(C*P**2, D)
        self.positional_embedding = nn.Embedding(N+1, D)
        self.classification = nn.Linear(N, N+1, bias=False)
        self.blocks = nn.Sequential(*[Block() for i in range(n_layers)])
        
    def forward(self, ix, targets = None):
        ix = ix @ self.embedding
        pass

        
            
class Block(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(D)
        self.ln1 = nn.LayerNorm(D)
        self.ln2 = nn.LayerNorm(D)

    def forward(self, ix):
        ix = ix + self.ffwd(self.ln1(ix))
        ix = ix + self.sa(self.ln2(ix))
        return ix
        
        
    
class Head(nn.Module):
    def __init__(self):
        super().__init__()
        self.key = torch.Linear()
        self.query = torch.Linear()
        self.value = torch.Linear()
        self.head_size = head_size
        
    def forward(self, ix): 
        k = self.key(ix)
        q = self.query(ix)
        
        wei = q @ k.transpose(-2, -1) * self.head_size**-0.5
        wei = F.softmax(wei, dim=-1)
        v = self.value(ix)

        out = wei @ v
        
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        